In [4]:
# load all data
import os
import polars as pl
data_dir = "./generated_all"
df = pl.DataFrame()
for name in os.listdir(data_dir):
    load_df = pl.read_parquet(os.path.join(data_dir, name))
    df = df.vstack(load_df)

print(df)


shape: (63_900, 80)
┌──────────┬──────────┬──────────┬──────────┬───┬──────────────────┬───────┬─────┬─────────────────┐
│ f1_min   ┆ f1_max   ┆ f1_std   ┆ f1_mean  ┆ … ┆ name_orig        ┆ music ┆ win ┆ name            │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆   ┆ ---              ┆ ---   ┆ --- ┆ ---             │
│ f64      ┆ f64      ┆ f64      ┆ f64      ┆   ┆ str              ┆ str   ┆ str ┆ str             │
╞══════════╪══════════╪══════════╪══════════╪═══╪══════════════════╪═══════╪═════╪═════════════════╡
│ 0.687108 ┆ 0.880667 ┆ 0.041199 ┆ 0.840509 ┆ … ┆ dance_data_96_co ┆ mMH0  ┆ 3   ┆ dance_data_96_c │
│          ┆          ┆          ┆          ┆   ┆ rrected          ┆       ┆     ┆ orrected        │
│ 0.556844 ┆ 0.880923 ┆ 0.064346 ┆ 0.834964 ┆ … ┆ dance_data_44_co ┆ mLO0  ┆ 2   ┆ dance_data_44_c │
│          ┆          ┆          ┆          ┆   ┆ rrected          ┆       ┆     ┆ orrected        │
│ 0.425007 ┆ 0.875003 ┆ 0.129454 ┆ 0.710535 ┆ … ┆ dance_data_31_co ┆ mW

In [ ]:
# Analysis 1 - compare generated features mean and standard deviation

index_df = df.group_by(
    by=["name",     ]
)